## To identify distinct transactional patterns and behavioral segments within the retail dataset(EDA)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_excel("../datasets/raw/Online Retail.xlsx")

In [4]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [19]:
print((df.isna().mean()*100).map('{:.2f}%'.format))

InvoiceNo       0.00%
StockCode       0.00%
Description     0.27%
Quantity        0.00%
InvoiceDate     0.00%
UnitPrice       0.00%
CustomerID     24.93%
Country         0.00%
dtype: object


In [15]:
print(df.duplicated().sum())

5268


In [8]:
# summary discription of numerical datas
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303



## 1. Numerica Data Analysis
### 1.1 Quantity Analysis
Tracks the volume of items per transaction.

- **Central Tendency:**  
  - Average purchase: **9.55 units**  
  - Median purchase: **3.0 units**  
  - Observation: Distribution is **right-skewed**, with many small purchases and a few very large ones.

- **Dispersion:**  
  - Standard deviation: **218.08**  
  - Maximum: **80,995 units**  
  - Indicates presence of **significant outliers**.

- **Anomalies:**  
  - Minimum: **-80,995 units**, likely representing **returns or cancellations**.

---

### 1.2 Temporal Range (InvoiceDate)
Analyzes the time span of the transactions.

- **Start Date:** `2010-12-01 08:26:00`  
- **End Date:** `2011-12-09 12:50:00`  
- **Average Date:** Around **July 4, 2011**  
- Observation: Data spans **approximately one year**, providing a complete seasonal view.

---

### 1.3 Unit Price Distribution
Pricing metrics for products in the catalog.

- **Pricing Tiers:**  
  - **25% (Budget):** ≤ $1.25  
  - **50% (Standard, Median):** $2.08  
  - **75% (Premium):** ≥ $4.13  

- **Note:**  
  - Minimum price: **-$11,062.06**, which likely represents **errors or refunds**.  
  - Data cleaning is recommended before financial modeling.

---

### 1.4. Customer Coverage
Insight into customer data quality.

- **Active Customers:** **406,829** records with valid `CustomerID`.  
- **Missing Data:** Approximately **135,080 rows** (~25%) lack `CustomerID`.  
- Observation: Missing data may impact **customer-level analysis** and **segmentation**.


In [10]:
print(df.describe(include='object').T)

              count unique                                 top    freq
InvoiceNo    541909  25900                              573585    1114
StockCode    541909   4070                              85123A    2313
Description  540455   4223  WHITE HANGING HEART T-LIGHT HOLDER    2369
Country      541909     38                      United Kingdom  495478


## 2. Categorical Feature Analysis

This section explores the non-numeric metadata of the transactional dataset. Initial observation reveals high cardinality in product identifiers and a heavy geographical bias toward the domestic market.

### 2.1 Feature Summary Table
The table below summarizes the distribution of the primary categorical variables across **541,909** records.

| Feature | Unique Count | Top Value | Frequency | Business Insight |
| :--- | :--- | :--- | :--- | :--- |
| **InvoiceNo** | 25,900 | `573585` | 1,114 | High frequency in single invoices suggests bulk/B2B purchasing. |
| **StockCode** | 4,070 | `85123A` | 2,313 | Inventory is diverse, but sales are concentrated in top SKUs. |
| **Description** | 4,223 | `WHITE HANGING HEART...` | 2,369 | This "Anchor Product" serves as a primary entry point for customers. |
| **Country** | 38 | `United Kingdom` | 495,478 | **91.4% Market Share** is domestic; international is a growth niche. |

---

### 2.2 Key Data Insights

#### A. The "Wholesaler" Signature
The presence of **Invoice 573585**, which contains **1,114 unique entries**, indicates that the dataset includes wholesale or B2B (Business-to-Business) entities. 
* **Model Impact:** These outliers can skew K-Means centroids. They are treated as a distinct "Wholesaler" segment to avoid diluting the "Retail Shopper" personas.

#### B. High Cardinality & Dimensionality
With over **4,000 unique product descriptions**, using standard encoding techniques (like One-Hot Encoding) would lead to the *Curse of Dimensionality*, expanding the dataset into 4,000+ sparse columns.
* **Modeling Strategy:** Instead of encoding product names, we engineered features such as **"Variety of Products Bought"** and **"Average Items per Transaction"** to capture behavior without overwhelming the model.



#### C. Geographical Concentration
The vast majority of transactions occur in the **United Kingdom (91.4%)**.
* **Strategic Action:** For marketing purposes, the "Country" feature is collapsed into a binary category: **Domestic (UK)** vs. **International**. This simplifies the model while retaining the most important geographical distinction.



---

### 2.3 Implications for Modeling
The categorical analysis confirms that the data is not "isotropic" (uniform). The high frequency of specific items and the dominance of the UK market suggest that a **Centroid-based approach (K-Means)** will be effective at slicing the main "Retail Cloud," while **DBSCAN** may be useful specifically for isolating the massive B2B invoices as outliers.

In [22]:
df = df.drop(columns=["CustomerID"])

In [23]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom


In [27]:
description_mode = df['Description'].mode()[0]
df["Description"] = df["Description"].fillna(description_mode)

In [29]:
print((df.isna().mean()*100).map('{:.2f}%'.format))

InvoiceNo      0.00%
StockCode      0.00%
Description    0.00%
Quantity       0.00%
InvoiceDate    0.00%
UnitPrice      0.00%
Country        0.00%
dtype: object
